# FACIAL EMOTION RECOGNITION

This model detects and classifies human emotions from facial expressions in both static images and live webcam feeds.

Libraries used: 
NumPy
OpenCV
Matplotlib
Tensorflow

Authors: ERC, IIT Bombay

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array


train_data_dir = 'train'
val_data_dir = 'test'

#printing some data
path=os.path.join(train_data_dir,"happy")
i=0

for img in os.listdir(path):
    i=i+1
    img_array=cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
    if (i<=3) :
        plt.imshow(img_array, cmap="gray")
        plt.show()
    else:
        break


#data preprocessing, augumenting the images to increase generalization of the model
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(48, 48), color_mode='grayscale', batch_size=64, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_data_dir, target_size=(48, 48), color_mode='grayscale', batch_size=32, class_mode='categorical')


In [ ]:
model = Sequential()

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, validation_data=val_generator, epochs=30)

In [ ]:
emotion_labels = [ 'Happy', 'Sad', 'Surprise']

#capturing videos from webcam
cap = cv2.VideoCapture(0)   

while True:
    
    #capturing a frame from the live webcam feed
    ret, frame = cap.read() 

    #ret=False means that there is some problem with capturing the frame
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
     # using haar cascade inbuilt model to detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = roi_gray.astype('float') / 255.0
        roi_gray = img_to_array(roi_gray)
        roi_gray = np.expand_dims(roi_gray, axis=0)
        
        emotion_prediction = model.predict(roi_gray)
        max_index = np.argmax(emotion_prediction)
        emotion = emotion_labels[max_index]
        print(emotion)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
    
    
    cv2.imshow('Emoji Emotion Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):   #press q to close the window
        break

cap.release()
cv2.destroyAllWindows()
